**External coupling** refers to a situation where a module relies on external systems, APIs, or data formats that it has little to no control over. This type of coupling can make code difficult to maintain or adapt when those external dependencies change.

### Example Scenario:
Imagine a system that integrates with an **external payment processing API**. The module in your code is tightly coupled to this external API, making it vulnerable to changes in the API’s structure, data format, or requirements.



### Code Example of External Coupling:



#### `payment_processor.py` (Tightly Coupled to External API)


In [1]:
import requests

class PaymentProcessor:
    def __init__(self, api_url, api_key):
        self.api_url = api_url
        self.api_key = api_key

    def process_payment(self, payment_details):
        # Directly using the structure expected by the external API
        payload = {
            'amount': payment_details['amount'],
            'currency': payment_details['currency'],
            'card_number': payment_details['card_number'],
            'card_expiry': payment_details['card_expiry'],
            'card_cvc': payment_details['card_cvc']
        }
        
        headers = {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }
        
        response = requests.post(self.api_url, json=payload, headers=headers)
        
        if response.status_code == 200:
            print("Payment processed successfully")
            return response.json()
        else:
            print("Failed to process payment")
            return response.text


### Explanation:
- **External Coupling**: The `PaymentProcessor` class is tightly coupled to the external payment processing API. It relies on the API's URL, data format (e.g., JSON structure), and authentication method (Bearer token).
- **Vulnerable to Changes**: If the API's structure changes (e.g., new required fields, different authentication headers, or changes in response format), the `PaymentProcessor` class will break or need significant updates.
- **Hard to Test**: Testing this class requires access to the external API, which can complicate unit tests and make them slower or less reliable.

### Problems with External Coupling:
1. **Maintenance Issues**: When the external API updates its requirements or data format, your code must be updated, which can lead to high maintenance costs.
2. **Testing Challenges**: You may need to use mocks or stubs to test your module, which can make tests more complex and less reliable.
3. **Adaptability**: If you decide to switch to a different payment provider, you need to refactor your code significantly to accommodate the new provider's format and requirements.

### Solution to Reduce External Coupling:
To reduce external coupling, introduce an **adapter pattern** that abstracts the external API and provides a uniform interface for the rest of the application. This way, if the external API changes, you only need to update the adapter.

#### Refactored Code Using an Adapter:

**`payment_adapter.py` (Adapter for the External API)**


In [ ]:
import requests

class PaymentAdapter:
    def __init__(self, api_url, api_key):
        self.api_url = api_url
        self.api_key = api_key

    def send_payment_request(self, payment_data):
        payload = self._format_payload(payment_data)
        headers = self._build_headers()

        response = requests.post(self.api_url, json=payload, headers=headers)

        return self._handle_response(response)

    def _format_payload(self, payment_data):
        # Maps internal data to external API's expected structure
        return {
            'amount': payment_data.amount,
            'currency': payment_data.currency,
            'card_number': payment_data.card_number,
            'card_expiry': payment_data.card_expiry,
            'card_cvc': payment_data.card_cvc
        }

    def _build_headers(self):
        return {
            'Authorization': f'Bearer {self.api_key}',
            'Content-Type': 'application/json'
        }

    def _handle_response(self, response):
        if response.status_code == 200:
            print("Payment processed successfully")
            return response.json()
        else:
            print("Failed to process payment")
            return response.text

class PaymentProcessor:
    def __init__(self, adapter):
        self.adapter = adapter

    def process_payment(self, payment_details):
        return self.adapter.send_payment_request(payment_details)

#from payment_adapter import PaymentAdapter
#from payment_processor import PaymentProcessor

# External API details
api_url = "https://api.paymentprovider.com/v1/payments"
api_key = "your_api_key_here"

# Create the adapter and processor
adapter = PaymentAdapter(api_url, api_key)
processor = PaymentProcessor(adapter)

# Payment details object (this would typically come from your app's logic)
payment_details = {
    'amount': 100.0,
    'currency': 'USD',
    'card_number': '4111111111111111',
    'card_expiry': '12/24',
    'card_cvc': '123'
}

# Process the payment
processor.process_payment(payment_details)


### Benefits of Using an Adapter:
1. **Single Point of Change**: If the external API changes, only the `PaymentAdapter` needs to be updated.
2. **Improved Testability**: You can mock the `PaymentAdapter` during testing without needing to hit the real external API.
3. **Flexibility**: If you switch to a different payment provider, you can implement a new adapter for the new provider without changing the `PaymentProcessor` logic.
4. **Reduced External Coupling**: The `PaymentProcessor` class no longer depends directly on the external API's details. It relies on the adapter, which abstracts those details.

This approach maintains a clean separation between your application logic and external dependencies, making your codebase more maintainable and adaptable to changes.